In [1]:
from google.colab import files
uploaded = files.upload()

Saving new_data.csv to new_data.csv


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
df = pd.read_csv("new_data.csv")
df

,lap_id,invalid_lap,BPS_SPEED,BPS_THROTTLE,BPS_STEER,BPS_BRAKE,BPS_LAPDISTANCE,BPS_WORLDPOSITIONX,BPS_WORLDPOSITIONY,BPS_WORLDFORWARDDIRX,...,APX2_YAW,APX2_PITCH,APX2_ROLL,APX2_left_dist,APX2_dist_apex_1,APX2_dist_apex_2,APX2_angle_to_apex1,APX2_angle_to_apex2,APX2_proj_from_ref,Target_CURRENTLAPTIMEINMS
0,10021698834789871149_1,1,316.0,1.000000,0.074568,0.0,223.0,274.071987,313.458779,-23579.0,...,-2.906792,0.002597,-0.015706,1.188303,101.216786,1.468423,176.387023,-73.297876,1.243211,15377
1,10021698834789871149_10,1,324.0,0.970802,0.000464,0.0,273.0,310.096901,278.846172,-23824.0,...,-2.975512,-0.004548,-0.010439,1.716472,101.368594,0.941933,178.312192,-68.980181,0.715854,12146
2,10021698834789871149_11,1,322.0,1.000000,-0.002375,0.0,258.0,298.743940,288.688005,-23746.0,...,-2.922204,-0.003019,-0.011373,2.731520,101.668646,0.122973,176.987177,53.851467,0.297512,12220
3,10021698834789871149_12,0,322.0,0.025177,0.006011,0.0,269.0,307.165908,281.569461,-23818.0,...,-2.923212,-0.002735,-0.009557,1.879479,101.417020,0.779785,177.377583,-68.683995,0.553155,12219
4,10021698834789871149_13,1,324.0,1.000000,0.021114,0.0,268.0,306.397540,282.215448,-23831.0,...,-2.824224,0.006822,-0.014098,4.818916,102.315233,2.167787,173.829692,99.915419,2.381417,12881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,9874269645241895165_2,1,144.0,0.000000,-0.525250,0.0,450.0,369.369267,134.915112,5165.0,...,-2.974651,-0.000140,-0.022406,2.409240,105.101572,7.066486,179.753790,-120.396863,5.033546,13686
821,9941972541231747669_1,1,324.0,1.000000,0.000000,0.0,263.0,302.615240,285.497492,-23572.0,...,-2.875317,-0.000517,-0.010140,1.617995,101.340908,1.039864,176.906931,-71.039304,0.814210,12490
822,9941972541231747669_2,0,324.0,1.000000,-0.000000,0.0,264.0,303.184481,284.646456,-23649.0,...,-2.914761,-0.002244,-0.008611,1.437935,101.289007,1.219306,177.457928,-71.360891,0.993987,12326
823,9948558370850722411_1,1,324.0,1.000000,-0.003344,0.0,276.0,301.987594,286.287858,-24030.0,...,-2.907833,0.001437,-0.014618,2.980371,101.744896,0.340118,175.711551,87.320007,0.545892,13604


In [16]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

target = "Target_CURRENTLAPTIMEINMS"

# Drop 'lap_id' and 'invalid_lap' as they are not numeric and not suitable for the model
X = df.drop(columns=["lap_id", "invalid_lap", target], errors="ignore")
y = df[target]

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define and train XGBoost model
model = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:gamma"
    # used gamma for now but can potentially log-transform the target and revert back to reg:squarederror
)

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"R² Score: {r2:.3f}")
print(f"RMSE: {rmse:.3f} ms")

R² Score: 0.403
RMSE: 3785.104 ms


In [5]:
mutual = ["APX2_SPEED", "STE_SPEED", "STM_SPEED", "THE_SPEED", "THS_SPEED", "BPS_SPEED",
          "APX1_SPEED", "APX1_YAW", "STE_THROTTLE", "APX2_WORLDFORWARDDIRY", "APX2_PITCH",
          "APX1_WORLDFORWARDDIRX", "THE_ext_TIMETOINMS", "THE_ext_LAPDISTANCE", "BPE_SPEED",
          "STS_ext_LAPDISTANCE", "STM_WORLDPOSITIONY", "STE_angle_to_apex2", "STM_THROTTLE"
          "STE_LAPDISTANCE", "APX2_THROTTLE", "STS_SPEED", "APX1_proj_from_ref",
          "STM_dist_apex_1", "STE_angle_to_apex1", "THE_LAPDISTANCE", "STM_LAPDISTANCE"
          "BPS_left_dist", "STE_WORLDFORWARDDIRX", "STE_PITCH", "APX1_left_dist",
          "STS_left_dist", "STS_dist_apex_2", "BPS_ROLL", "STS_WORLDPOSITIONY", "STM_WORLDPOSITIONX"
          "THS_PITCH", "STS_BRAKE", "STM_YAW", "STE_WORLDFORWARDDIRY", "THS_WORLDFORWARDDIRY", "STS_ext_TIMETOINMS"
          "APX1_WORLDPOSITIONX", "THE_YAW", "STE_ROLL", "STS_proj_from_ref", "THE_WORLDFORWARDDIRX", "STM_WORLDFORWARDDIRX"
          "STE_YAW", "STE_WORLDPOSITION"]

info_gain = ["APX2_SPEED", "THS_SPEED", "BPS_SPEED", "STM_SPEED", "APX1_SPEED",
             "BPE_angle_to_apex2", "THE_ext_TIMETOINMS", "STE_proj_from_ref",
             "THE_SPEED", "THS_ext_LAPDISTANCE", "APX2_STEER", "BPS_left_dist",
             "BPS_ext_TIMETOINMS", "STM_ROLL", "STS_SPEED", "STE_STEER", "BPS_STEER",
             "THS_WORLDPOSITIONY", "THS_proj_from_ref", "STM_LAPDISTANCE", "APX2_angle_to_apex1",
             "THE_ext_LAPDISTANCE", "BPE_left_dist", "BPE_WORLDFORWARDDIRY", "THE_dist_apex_2",
             "STM_WORLDPOSITIONX", "APX2_angle_to_apex2", "THS_STEER", "THS_THROTTLE",
             "THE_proj_from_ref", "BPS_WORLDPOSITIONX", "APX1_STEER"]

elastic = ["THE_BRAKE", "STM_BRAKE", "BPE_ROLL", "STM_ROLL", "BPS_SPEED", "THE_SPEED", "STM_SPEED", "STE_SPEED",
           "APX2_SPEED", "BPS_STEER", "STS_STEER", "STM_STEER", "APX1_SPEED", "APX2_SPEED", "STS_THROTTLE", "STM_THROTTLE", "STE_THROTTLE"
           "THS_WORLDFORWARDDIRX", "APX2_WORLDFORWARDDIRY", "STM_WORLDPOSITIONX", "APX1_WORLDPOSITIONY", "BPE_YAW", "THS_YAW", "STS_YAW"
           "APX2_YAW", "BPE_angle_to_apex1", "STS_angle_to_apex1", "APX2_angle_to_apex1", "STS_angle_to_apex2", "STM_angle_to_apex2"
           "STE_angle_to_apex2", "BPE_dist_apex_1", "THE_dist_apex_1", "STM_dist_apex_1", "STE_dist_apex_1", "THS_ext_LAPDISTANCE",
           "THE_ext_LAPDISTANCE", "STE_ext_LAPDISTANCE", "BPS_ext_TIMETOINMS", "THS_ext_TIMETOINMS", "THE_ext_TIMETOINMS", "STS_ext_TIMETOINMS",
           "STE_ext_TIMETOINMS", "STS_left_dist", "STM_left_dist", "BPE_proj_from_ref"]

xgboost = ["STM_SPEED", "STE_SPEED", "STS_ext_LAPDISTANCE", "APX2_SPEED", "BPS_YAW", "STM_THROTTLE", "STS_LAPDISTANCE", "STS_LAPDISTANCE",
           "STM_WORLDFORWARDDIRX", "BPE_angle_to_apex2", "APX2_angle_to_apex2", "APX2_dist_apex_1", "STE_WORLDFORWARDDIRY",
           "STS_ext_TIMETOINMS", "STM_WORLDFORWARDDIRY", "STE_THROTTLE", "STM_angle_to_apex1", "APX2_WORLDFORWARDDIRY", "STS_SPEED", "THS_SPEED"
           "APX1_left_dist", "STE_ext_TIMETOINMS", "THS_WORLDFORWARDDIRY", "THS_ext_LAPDISTANCE", "STS_YAW", "APX1_dist_apex_1", "STM_BRAKE",
           "THS_ROLL", "STS_angle_to_apex1", "THS_proj_from_ref", "STE_YAW", "APX1_angle_to_apex2", "STM_ROLL", "STE_LAPDISTANCE",
           "THE_ext_TIMETOINMS", "BPS_THROTTLE", "APX1_WORLDPOSITIONX", "APX2_WORLDPOSITIONY", "BPS_left_dist", "BPE_LAPDISTANCE",
           "BPS_angle_to_apex1", "APX2_PITCH", "BPE_STEER", "APX2_YAW", "THS_WORLDFORWARDDIRX", "STS_WORLDFORWARDDIRY", "STE_PITCH",
           "STS_PITCH", "APX1_STEER", "BPE_WORLDFORWARDDIRX", "THS_STEER"]

random_forest = ["STM_SPEED", "STE_SPEED", "APX2_SPEED", "STS_angle_to_apex1", "STS_YAW", "STM_angle_to_apex1", "STE_ext_TIMETOINMS",
                 "THE_ext_TIMETOINMS", "THS_SPEED", "BPS_SPEED", "STM_WORLDFORWARDDIRX", "APX2_PITCH", "STM_WORLDFORWARDDIRY",
                 "STE_WORLDFORWARDDIRY", "THE_WORLDFORWARDDIRX", "BPE_YAW", "APX2_angle_to_apex2", "THS_STEER", "THE_ext_LAPDISTANCE",
                 "STS_SPEED", "STM_YAW", "STE_PITCH", "STE_WORLDFORWARDDIRX", "THE_dist_apex_1", "THS_proj_from_ref", "BPS_YAW",
                 "THE_ROLL", "BPE_WORLDFORWARDDIRY", "BPS_THROTTLE", "APX1_SPEED", "THE_SPEED", "APX2_STEER", "BPS_PITCH", "APX1_BRAKE",
                 "STS_WORLDPOSITIONX", "BPE_ext_TIMETOINMS", "THS_PITCH", "STS_WORLDFORWARDDIRX", "THS_ext_TIMETOINMS"]


In [6]:
union = set(mutual) | set(xgboost) | set(random_forest) | set(elastic) | set(info_gain)
print(sorted(union))

['APX1_BRAKE', 'APX1_SPEED', 'APX1_STEER', 'APX1_WORLDFORWARDDIRX', 'APX1_WORLDPOSITIONX', 'APX1_WORLDPOSITIONY', 'APX1_YAW', 'APX1_angle_to_apex2', 'APX1_dist_apex_1', 'APX1_left_dist', 'APX1_proj_from_ref', 'APX2_PITCH', 'APX2_SPEED', 'APX2_STEER', 'APX2_THROTTLE', 'APX2_WORLDFORWARDDIRY', 'APX2_WORLDPOSITIONY', 'APX2_YAW', 'APX2_angle_to_apex1', 'APX2_angle_to_apex2', 'APX2_dist_apex_1', 'BPE_LAPDISTANCE', 'BPE_ROLL', 'BPE_SPEED', 'BPE_STEER', 'BPE_WORLDFORWARDDIRX', 'BPE_WORLDFORWARDDIRY', 'BPE_YAW', 'BPE_angle_to_apex1', 'BPE_angle_to_apex2', 'BPE_dist_apex_1', 'BPE_ext_TIMETOINMS', 'BPE_left_dist', 'BPE_proj_from_ref', 'BPS_PITCH', 'BPS_ROLL', 'BPS_SPEED', 'BPS_STEER', 'BPS_THROTTLE', 'BPS_WORLDPOSITIONX', 'BPS_YAW', 'BPS_angle_to_apex1', 'BPS_ext_TIMETOINMS', 'BPS_left_dist', 'STE_LAPDISTANCE', 'STE_PITCH', 'STE_ROLL', 'STE_SPEED', 'STE_STEER', 'STE_THROTTLE', 'STE_THROTTLETHS_WORLDFORWARDDIRX', 'STE_WORLDFORWARDDIRX', 'STE_WORLDFORWARDDIRY', 'STE_WORLDPOSITION', 'STE_YAW', 'STE

In [13]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

another_selected_features = [
    "APX2_SPEED", "STM_SPEED", "STE_ext_TIMETOINMS", "STS_angle_to_apex1",
    "APX2_STEER", "THS_proj_from_ref", "STE_THROTTLE", "APX1_SPEED",
    "STM_angle_to_apex2", "STE_proj_from_ref", "THE_ROLL", "STE_ext_LAPDISTANCE",
    "STS_ext_TIMETOINMS", "THE_SPEED", "BPS_angle_to_apex1", "STM_YAW",
    "STE_STEER", "APX1_YAW", "APX1_WORLDFORWARDDIRX", "APX2_THROTTLE",
    "APX1_proj_from_ref", "STE_angle_to_apex1", "BPS_ROLL", "STS_BRAKE",
    "THE_YAW", "STE_ROLL", "STS_proj_from_ref", "THE_WORLDFORWARDDIRX",
    "BPE_angle_to_apex1", "BPE_WORLDFORWARDDIRY", "APX2_angle_to_apex1", "THS_ROLL",
    "BPE_proj_from_ref", "THS_THROTTLE", "BPS_PITCH", "BPS_THROTTLE",
    "STS_angle_to_apex2", "THE_proj_from_ref", "APX1_BRAKE", "BPE_angle_to_apex2",
    "THS_YAW", "BPE_YAW", "APX2_angle_to_apex2", "BPE_ROLL",
    "BPS_YAW", "BPE_ext_TIMETOINMS", "APX1_STEER", "APX1_angle_to_apex2",
    "BPS_STEER", "STM_angle_to_apex1", "STM_left_dist"
]

# Data preparation
X = df[another_selected_features]
y = df["Target_CURRENTLAPTIMEINMS"]

# Log-transformation to stabilize variance
y_log = np.log1p(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

# Model training
model = XGBRegressor(
    objective="reg:squarederror",  # can go back to reg:gamma depending on model performance
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)
model.fit(X_train, y_train)

# Predict and reverse log transformation
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Evaluation metrics
r2 = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"R² Score: {r2:.3f}")
print(f"RMSE: {rmse:.3f} ms")

R² Score: 0.401
RMSE: 3793.470 ms


In [25]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
# Data preparation
y = df["Target_CURRENTLAPTIMEINMS"]
y_log = np.log1p(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)
parameters = {
    "subsample": 0.5,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
    "n_estimators": 800,
    "min_child_weight": 5,
    "max_depth": 3,
    "learning_rate": 0.02,
    "colsample_bytree": 0.9,
}

model = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    **parameters
)

model.fit(X_train, y_train)

# Predict and reverse log transformation
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Evaluation metrics
r2 = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
# Training RMSE
train_pred = np.expm1(model.predict(X_train))
train_true = np.expm1(y_train)
train_rmse = np.sqrt(mean_squared_error(train_true, train_pred))

print(f"Train RMSE: {train_rmse:.3f} ms")
print(f"Test RMSE: {rmse:.3f} ms")
print(f"R^2 (test): {r2:.4f}")

# Predictor values
i = 0
sample = X_test.iloc[i]
pred_value = y_pred[i]
print(f"\nActual Target_CURRENTLAPTIMEINMS: {np.expm1(y_test.iloc[i]):,.2f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {pred_value:,.3f}")

Train RMSE: 334.569 ms
Test RMSE: 3582.276 ms
R^2 (test): 0.4657

Actual Target_CURRENTLAPTIMEINMS: 12,272.00
Predicted Target_CURRENTLAPTIMEINMS: 12,538.941


In [23]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Define model and parameter grid
xgb_model = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
)

param_grid = {
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.02, 0.03],
    'n_estimators': [300, 500, 800],
    'subsample': [0.5, 0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_weight': [1, 3, 5, 7],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2],
}

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,                # number of random combinations to try
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

# Print results
print("\nBest Parameters Found:")
for key, val in random_search.best_params_.items():
    print(f"  {key}: {val}")

print(f"\nBest Cross-Validation RMSE: {-random_search.best_score_:.3f}")

# Evaluate on test set
best_model = random_search.best_estimator_
y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

test_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"\nTest RMSE: {test_rmse:.3f} ms")
print(f"R² on Test Set: {r2:.4f}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits

Best Parameters Found:
  subsample: 0.5
  reg_lambda: 1
  reg_alpha: 0
  n_estimators: 800
  min_child_weight: 5
  max_depth: 3
  learning_rate: 0.02
  colsample_bytree: 0.9

Best Cross-Validation RMSE: 0.090

Test RMSE: 3582.276 ms
R² on Test Set: 0.4657


In [26]:
import numpy as np
import pandas as pd

def suggest_best_setup_for_log_model(
    model,
    X_train: pd.DataFrame,
    percentiles=(0.05, 0.95),
    N=50_000,
    top_k=5,
    seed=42,
    hard_bounds: dict | None = None,   # e.g. {"BPS_THROTTLE": (0,1), "BPS_BRAKE": (0,1)}
    int_cols: list[str] | None = None, # e.g. ["gear"]
):
    rng = np.random.default_rng(seed)

    # Feature-wise bounds from TRAIN data (avoid leakage)
    num_cols = X_train.select_dtypes(include=["number"]).columns
    q = X_train[num_cols].quantile([percentiles[0], percentiles[1]])
    low, high = q.loc[percentiles[0]], q.loc[percentiles[1]]

    # Samples candidates uniformly within bounds
    candidates = {
        c: rng.uniform(low=low[c], high=high[c], size=N) for c in num_cols
    }
    Xcand = pd.DataFrame(candidates)[num_cols]

    # Enforce domain hard-bounds (0–1 controls)
    if hard_bounds:
        for c, (lo, hi) in hard_bounds.items():
            if c in Xcand.columns:
                Xcand[c] = np.clip(Xcand[c], lo, hi)

    # Forces integer columns (e.g., gear)
    if int_cols:
        for c in int_cols:
            if c in Xcand.columns:
                Xcand[c] = Xcand[c].round().astype(int)

    # Predicts (model trained on log1p), then invert to ms
    ypred_log = model.predict(Xcand)
    ypred_ms = np.expm1(ypred_log)

    # Picks best lap times
    order = np.argsort(ypred_ms)
    best_idx = order[:top_k]

    topk = Xcand.iloc[best_idx].copy()
    topk["Predicted_Target_CURRENTLAPTIMEINMS"] = ypred_ms[best_idx]

    best_setup = topk.iloc[0].drop("Predicted_Target_CURRENTLAPTIMEINMS")
    best_pred = float(topk.iloc[0]["Predicted_Target_CURRENTLAPTIMEINMS"])

    return topk.reset_index(drop=True), best_setup, best_pred

topk, best_setup, best_pred = suggest_best_setup_for_log_model(
    model=model,
    X_train=X_train,
    percentiles=(0.05, 0.95),
    N=50_000,
    top_k=5,
    seed=42,
    hard_bounds={
        # "BPS_THROTTLE": (0.0, 1.0),
        # "BPS_BRAKE": (0.0, 1.0),
        # "BPS_STEER": (-1.0, 1.0),
        # "BPS_SPEED": (0.0, None),
    },
    int_cols=[
        # "gear",
    ],
)

print("\n=== Suggested optimal setup (within 5–95% train ranges) ===")
for k, v in best_setup.items():
    print(f"{k}: {v:.4f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {best_pred:,.3f} ms")

print("\nTop-5 suggested setups:")
print(topk.to_string(index=False))



=== Suggested optimal setup (within 5–95% train ranges) ===
BPS_SPEED: 315.1295
BPS_THROTTLE: 0.6863
BPS_STEER: 0.0363
BPS_BRAKE: 0.0000
BPS_LAPDISTANCE: 302.8198
BPS_WORLDPOSITIONX: 320.4783
BPS_WORLDPOSITIONY: 320.1430
BPS_WORLDFORWARDDIRX: -23550.7962
BPS_WORLDFORWARDDIRY: 64.4002
BPS_YAW: -2.3208
BPS_PITCH: 0.0060
BPS_ROLL: -0.0160
BPS_left_dist: 2.7905
BPS_dist_apex_1: 97.0269
BPS_dist_apex_2: 254.0513
BPS_angle_to_apex1: 39.7190
BPS_angle_to_apex2: 25.1985
BPS_proj_from_ref: 1.7091
BPS_ext_LAPDISTANCE: 316.2646
BPS_ext_TIMETOINMS: 113.0390
BPE_SPEED: 207.4407
BPE_THROTTLE: 0.1215
BPE_STEER: 0.2650
BPE_BRAKE: 0.9589
BPE_LAPDISTANCE: 287.5714
BPE_WORLDPOSITIONX: 345.6498
BPE_WORLDPOSITIONY: 250.9638
BPE_WORLDFORWARDDIRX: -23680.5830
BPE_WORLDFORWARDDIRY: -389.7572
BPE_YAW: -2.4293
BPE_PITCH: 0.0033
BPE_ROLL: 0.0060
BPE_left_dist: 13.9320
BPE_dist_apex_1: 7.7295
BPE_dist_apex_2: 144.5176
BPE_angle_to_apex1: 36.1805
BPE_angle_to_apex2: 22.3004
BPE_proj_from_ref: 3.9320
BPE_ext_LAPDI